In [1]:
!pip install -q trl math_verify evaluate vllm==0.8.2 unsloth

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r "/content/drive/MyDrive/Graduate Project/MATH_train_staging.csv" .
!cp -r "/content/drive/MyDrive/Graduate Project/MATH_val_staging.csv" .

In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from IPython.display import display, Markdown
from trl import GRPOConfig, GRPOTrainer
from math_verify import parse, verify
from datasets import Dataset
from pprint import pprint
from tqdm import tqdm

import pandas as pd
import evaluate
import random
import torch
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-05 20:07:16 [__init__.py:239] Automatically detected platform cuda.


In [5]:
SEED = 42
random.seed(SEED)
_ = torch.manual_seed(SEED)
DEVICE = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else (
        torch.device("mps")
        if torch.backends.mps.is_available()
        else torch.device("cpu")
    )
)
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
MAX_NEW_TOKENS = 10000
MAX_SEQ_LEN = 512
LORA_RANK = 16
NUM_EPOCHS = 2
BLEU = evaluate.load("bleu")

In [6]:
train_data = pd.read_csv("MATH_train_staging.csv")
val_data = pd.read_csv("MATH_val_staging.csv")

In [7]:
train_data["extracted_answer"] = train_data["reasoning"].map(lambda x: parse(x)[1])
val_data["extracted_answer"] = val_data["reasoning"].map(lambda x: parse(x)[1])
for split_str in [
    "-OR-",
    " OR ",
    "OR\n",
    "\\$\\$OR \\$\\$",
    "\\\\text\\{OR\\}",
    "\\\\textbf\\{OR\\}",
    "\\\\centerline\\{\\\\bf \\{OR\\}\\}",
    "\\\\centerline\\{\\{\\\\bf OR\\}\\}",
]:
    train_data.loc[
        train_data["reasoning"].str.split(split_str).str.len() > 1, "reasons"
    ] = train_data[train_data["reasoning"].str.split(split_str).str.len() > 1][
        "reasoning"
    ].str.split(
        split_str
    )
    val_data.loc[
        val_data["reasoning"].str.split(split_str).str.len() > 1, "reasons"
    ] = val_data[val_data["reasoning"].str.split(split_str).str.len() > 1][
        "reasoning"
    ].str.split(
        split_str
    )
train_data.loc[train_data["reasons"].isna(), "reasons"] = train_data.loc[
    train_data["reasons"].isna()
]["reasoning"].str.split("-OR-")
val_data.loc[val_data["reasons"].isna(), "reasons"] = val_data.loc[
    val_data["reasons"].isna()
]["reasoning"].str.split("-OR-")

In [8]:
display(Markdown("Question -> " + train_data.loc[0]["question_text"]))
display(Markdown("Reasoning -> " + train_data.loc[0]["reasons"][0]))
display(Markdown("Answer -> $" + train_data.loc[0]["extracted_answer"] + "$"))

Question -> Find the equation whose graph is a parabola with vertex $(2,4)$, vertical axis of symmetry, and contains the point $(1,1)$. Express your answer in the form "$ax^2+bx+c$".

Reasoning -> Since the axis of symmetry is vertical and the vertex is $(2,4)$, the parabola may also be written as  \[y=a(x-2)^2+4\] for some value of $a$.  Plugging the point $(1,1)$ into this expression gives  \[1=a(1-2)^2+4=a+4.\] This tells us $a=-3$.

Our equation is  \[y=-3(x-2)^2+4.\] Putting it $y=ax^2+bx+c$ form requires expanding the square, so we get  \[y=-3(x^2-4x+4)+4=\boxed{-3x^2+12x-8}.\]

Answer -> $-3x^2+12x-8$

In [9]:
train_dataset = Dataset.from_pandas(train_data[["dataset_id", "question_id", "question_text", "reasons", "extracted_answer"]])
val_dataset = Dataset.from_pandas(val_data[["dataset_id", "question_id", "question_text", "reasons", "extracted_answer"]])

In [10]:
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. "
    "The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. "
    "The Assistant thinks and reasons about the question in a step-by-step manner, ensuring the steps follow a logical progression. "
    "If the question asks for a number or expression, the Assistant encloses it within the boxed latex command inside the answer. "
    "The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> step-by-step reasoning process here </think><answer> answer here </answer>"
)

In [11]:
def make_conversation(example):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["question_text"]},
        ]
    }

In [12]:
train_dataset = train_dataset.map(make_conversation).remove_columns(["question_text"])
val_dataset = val_dataset.map(make_conversation).remove_columns(["question_text"])

Map:   0%|          | 0/5988 [00:00<?, ? examples/s]

Map:   0%|          | 0/1497 [00:00<?, ? examples/s]

In [13]:
llm, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = 4*MAX_SEQ_LEN,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = LORA_RANK,
    gpu_memory_utilization = 0.7,
    random_state = SEED,
)
model = FastLanguageModel.get_peft_model(
    llm,
    r = LORA_RANK,
    target_modules = ["gate_proj", "up_proj", "down_proj",],
    lora_alpha = LORA_RANK,
    use_gradient_checkpointing = "unsloth",
    random_state = SEED,
)
model.print_trainable_parameters()

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 69.2%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 24.95 GB. Also swap space = 6 GB.
INFO 05-05 20:08:01 [config.py:585] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
WARNING 05-05 20:08:01 [ar

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.7 patched 28 layers with 0 QKV layers, 0 O layers and 28 MLP layers.


trainable params: 15,138,816 || all params: 3,227,888,640 || trainable%: 0.4690


In [14]:
def format_reward(completions, **kwargs):
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<think>.*?</think>\s*<answer>.*?</answer>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content) for content in completion_contents]
    return [1.0 if match else 0.0 for match in matches]

In [15]:
def accuracy_reward(completions, **kwargs):
    """Reward function that checks if the answer in completion has the ground truth and is equal."""
    solutions = kwargs["extracted_answer"]
    pattern = r"^<think>(.*?)</think>\s*<answer>(.*?)</answer>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    rewards = []
    for content, solution in zip(completion_contents, solutions):
        gold_parsed = solution
        matches = re.search(pattern, content)
        if matches:
            if len(matches.groups()) == 2:
                answer_parsed = parse(matches.group(2))
            else:
                answer_parsed = None
        else:
            answer_parsed = None
        if len(gold_parsed) != 0:
            try:
                if answer_parsed is None:
                    rewards.append(0.0)
                else:
                    rewards.append(float(verify(answer_parsed, gold_parsed)))
            except Exception:
                rewards.append(0.0)
        else:
            rewards.append(1.0)
    return rewards

In [16]:
def similarity_reward(completions, **kwargs):
    """Reward function that ensures the reasoning is similar to the ground truth."""
    reasons = kwargs["reasons"]
    pattern = r"^<think>(.*?)</think>\s*<answer>(.*?)</answer>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    rewards = []
    for content, reason in zip(completion_contents, reasons):
        matches = re.search(pattern, content)
        if matches:
            if len(matches.groups()) >= 1:
                reason_parsed = matches.group(1)
            else:
                reason_parsed = None
        else:
            reason_parsed = None
        if reason_parsed is None:
            rewards.append(0.0)
        else:
            try:
                bleu = BLEU.compute(predictions=[reason_parsed], references=[reason])
                rewards.append(bleu["bleu"])
            except Exception:
                rewards.append(0.0)
    return rewards

In [17]:
training_args = GRPOConfig(
    use_vllm=True,
    output_dir=f"/content/drive/MyDrive/Graduate Project/{MODEL_NAME.split('/')[1]}-CoT",
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    remove_unused_columns=False,
    gradient_accumulation_steps=4,
    num_train_epochs=NUM_EPOCHS,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_checkpointing=True,
    max_completion_length=MAX_NEW_TOKENS,
    num_generations=8,
    max_prompt_length=4*MAX_SEQ_LEN,
    max_grad_norm = 0.1,
    report_to=["tensorboard"],
    logging_steps=10,
    push_to_hub=False,
    save_strategy="steps",
    save_steps=10,
)

In [18]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[format_reward, accuracy_reward, similarity_reward],
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer
)

In [19]:
trainer.train(resume_from_checkpoint=True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,988 | Num Epochs = 2 | Total steps = 2,994
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 15,138,816/3,000,000,000 (0.50% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / format_reward,rewards / accuracy_reward,rewards / similarity_reward
2760,1.083800,1.305704,0.524491,436.990625,27.094977,0.846875,0.356250,0.102579
2770,0.003500,1.371176,0.534367,409.081250,0.086740,0.896875,0.350000,0.124301
2780,0.004900,1.499496,0.506613,390.543750,0.123287,0.903125,0.465625,0.130746
2790,0.005700,1.352458,0.503317,398.637500,0.143440,0.890625,0.346875,0.114958


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / format_reward,rewards / accuracy_reward,rewards / similarity_reward
2760,1.083800,1.305704,0.524491,436.990625,27.094977,0.846875,0.356250,0.102579
2770,0.003500,1.371176,0.534367,409.081250,0.086740,0.896875,0.350000,0.124301
2780,0.004900,1.499496,0.506613,390.543750,0.123287,0.903125,0.465625,0.130746
2790,0.005700,1.352458,0.503317,398.637500,0.143440,0.890625,0.346875,0.114958
2800,0.003900,1.354906,0.504729,425.431250,0.096513,0.856250,0.378125,0.120531
2810,0.005100,1.385944,0.577236,477.356250,0.126906,0.862500,0.415625,0.107819
2820,0.005900,1.424763,0.583068,413.350000,0.146830,0.890625,0.384375,0.149763
2830,0.013900,1.392366,0.531482,454.196875,0.347792,0.871875,0.396875,0.123616
2840,0.003800,1.332091,0.568039,450.300000,0.096004,0.846875,0.375000,0.110216
2850,0.004800,1.350810,0.595192,461.931250,0.119206,0.856250,0.350000,0.144560


TrainOutput(global_step=2994, training_loss=0.004150942405318728, metrics={'train_runtime': 18839.9612, 'train_samples_per_second': 0.636, 'train_steps_per_second': 0.159, 'total_flos': 0.0, 'train_loss': 0.004150942405318728})

In [20]:
trainer.save_model(f"/content/drive/MyDrive/Graduate Project/{MODEL_NAME.split('/')[1]}-CoT-Reasoning")